In [ ]:
import os
import re
import sqlite3
import json
from urllib.parse import urlparse
from collections import Counter

# Function to extract third-party requests from a single entry
def extract_third_party_requests(entry, host):
    request = entry.get('request', {})
    if 'url' in request:
        request_url_parts = urlparse(request['url']).hostname.rsplit('.')
        if (request_url_parts[-1] == 'uk' and request_url_parts[-2] == 'co'):
            request_url = request_url_parts[-3] + '.' + request_url_parts[-2] + '.' + request_url_parts[-1]
        else:
            request_url = request_url_parts[-2] + '.' + request_url_parts[-1]

        if '_resourceType' in entry:
            if host not in request_url and entry['_resourceType'] not in ['stylesheet', 'font', 'image']:
                return request_url
            
        else:
            if host not in request_url:
                return request_url

    return None

# Function to process HAR files in a folder
def process_har_folder(folder_path):
    all_request_domains = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.har'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                har_data = json.load(file)
                entries = har_data.get('log', {}).get('entries', [])
                for entry in entries:
                    host_match = re.search(r'(m|www)\.(.*)(\.har)', filename)
                    host = host_match.group(2) if host_match else None
                    
                    domain = extract_third_party_requests(entry, host)
                    if domain:
                        all_request_domains.append(domain)
    return all_request_domains

# Database initialization
db_conn = sqlite3.connect('third_party_analysis.db')
db_cursor = db_conn.cursor()

# Create a table to store domain counts
db_cursor.execute('''
    CREATE TABLE IF NOT EXISTS domain_counts (
        domain TEXT PRIMARY KEY,
        count INTEGER
    )
''')

folder_paths = ['../archives/news/desktop/', '../archives/news/mobile/']

# Initialize the list before the loop
all_request_domains = []

for folder_path in folder_paths:
    # Process each folder and update the list of domains
    all_request_domains += process_har_folder(folder_path)

# Use Counter to count the occurrences of each domain
domain_counter = Counter(all_request_domains)

# Insert domain counts into the SQLite database
for domain, count in domain_counter.items():
    db_cursor.execute('INSERT OR REPLACE INTO domain_counts VALUES (?, ?)', (domain, count))

# Commit the changes and close the database connection
db_conn.commit()
db_conn.close()

print(f"Total number of unique domains: {len(domain_counter)}")


In [ ]:
db_conn = sqlite3.connect('third_party_analysis.db')
db_cursor = db_conn.cursor()

# Execute the query to retrieve and order the records
db_cursor.execute('SELECT domain, count FROM domain_counts ORDER BY count DESC')
sorted_records = db_cursor.fetchall()

# Print the sorted records
print("Count\tThird party domain")
print("------------------------------------")
for domain, count in sorted_records:
    print(f"{count}\t{domain}")

# Close the database connection
db_conn.close()
